In [1]:
import cobra
import pandas as pd
from collections import OrderedDict

def build_reaction_equation_from_metabolites_dict(met_dict, arrow='<=>', floatdecimal=6):
    lhs = []; rhs = [];
    for k,v in met_dict.items():
        v = float(v)
        if v == -1:
            lhs.append(k)
        elif v == 1:
            rhs.append(k)
        elif v < 0 and v != -1 and v.is_integer():
            lhs.append(' '.join([str(-int(v)), k]))
        elif v > 0 and v != 1 and v.is_integer():
            rhs.append(' '.join([str(int(v)), k]))
        elif v < 0 and v != -1:
            lhs.append(' '.join([('{:.' + str(floatdecimal) + 'f}').format(-v), k]))
        elif v > 0 and v != 1:
            rhs.append(' '.join([('{:.' + str(floatdecimal) + 'f}').format(v), k]))
    return ' '.join([ ' + '.join(lhs), arrow, ' + '.join(rhs)])

#### Literature
Anaerobic near growth-arrest glucose uptake: Boender et al., 2009. pmid:19592533<br>
vGlc = 0.5 mmol Glc / gDW / h<br>
Aerobic near growth-arrest glucose uptake: Vos et al., 2016. pmid:27317316<br>
vGlc = 0.039 mmol Glc / gDW / h<br>
N-lim and P-lim growth-arrest data: Liu et al., 2019. pmid:31375494<br>

In [2]:
model = cobra.io.load_json_model('../../build_model/input/GSM_iSace1144_rba.json')
model.solver = 'cplex'

Using license file /home/hvdinh16/Workspace/Softwares/gurobi910/linux64/gurobi.lic
Academic license - for non-commercial use only - expires 2023-07-30


In [3]:
# Remove GAM from biomass equation
for biom in ['BIOMASS_AERO_SC_hvd', 'BIOMASS_BATCHANAERO_SC_hvd']:
    adp = model.metabolites.adp_c
    gam0 = model.reactions.get_by_id(biom).metabolites[adp]
    met_dict = OrderedDict({met.id:v for met,v in model.reactions.get_by_id(biom).metabolites.items()})
    for met in ['atp_c', 'h2o_c']:
        met_dict[met] += gam0
    for met in ['adp_c', 'pi_c', 'h_c']:
        met_dict[met] -= gam0
    biom_eqn = build_reaction_equation_from_metabolites_dict(met_dict, arrow='-->')
    model.reactions.get_by_id(biom).reaction = biom_eqn

In [4]:
# Aerobic
with model as m:
    m.reactions.EX_glc__D_e.bounds = (-0.039, -0.039)
    m.reactions.EX_o2_e.bounds = (-1000,1000)
    m.reactions.ATPM_c.bounds = (0,1000)
    m.reactions.FADH2t_c_m.knock_out()

    m.objective = dict()
    m.reactions.ATPM_c.objective_coefficient = 1
    fba = m.optimize()
    print('ATPM_c', fba['ATPM_c'])

ATPM_c 0.9983999999999997


In [5]:
# Anaerobic
with model as m:
    m.reactions.EX_glc__D_e.bounds = (-0.5, -0.5)
    m.reactions.EX_o2_e.bounds = (0,1000)
    m.reactions.ATPM_c.bounds = (0,1000)
    m.reactions.FADH2t_c_m.knock_out()

    m.objective = dict()
    m.reactions.ATPM_c.objective_coefficient = 1
    fba = m.optimize()
    print('ATPM_c', fba['ATPM_c'])

ATPM_c 1.0


In [6]:
# Nitrogen-limited
with model as m:
    m.reactions.EX_glc__D_e.bounds = (-0.57, -0.57)
    m.reactions.EX_o2_e.bounds = (-0.71, -0.71)
    m.reactions.EX_etoh_e.bounds = (0.75, 1000)
    m.reactions.EX_co2_e.bounds = (1.5,1000)
    m.reactions.ATPM_c.bounds = (0,1000)
    m.reactions.FADH2t_c_m.knock_out()

    m.objective = dict()
    m.reactions.ATPM_c.objective_coefficient = 1
    fba = m.optimize()
    print('ATPM_c', fba['ATPM_c'])

ATPM_c 3.9326666666666665


In [7]:
# Phosphate-limited
with model as m:
    m.reactions.EX_glc__D_e.bounds = (-0.53, -0.53)
    m.reactions.EX_o2_e.bounds = (-0.94, -0.94)
    m.reactions.EX_etoh_e.bounds = (0.55, 1000)
    m.reactions.EX_co2_e.bounds = (1.6,1000)
    m.reactions.ATPM_c.bounds = (0,1000)
    m.reactions.FADH2t_c_m.knock_out()

    m.objective = dict()
    m.reactions.ATPM_c.objective_coefficient = 1
    fba = m.optimize()
    print('ATPM_c', fba['ATPM_c'])

ATPM_c 4.757333333333332
